In [ ]:
# CNN_LSTM_train.ipynb

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchvision import transforms

from CNN_LSTM_dataset import HelplessnessVideoDataset
from CNN_LSTM_model import HelplessnessClassifier


In [2]:
### A) Device Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [ ]:
# B) Data Directories & Base Datasets

project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), '..', '..'))

train_dir = os.path.join(project_root, "data", "train")
val_dir   = os.path.join(project_root, "data", "val")

In [4]:
# C) Define Grayscale Augmentation Transforms

# For training, we add random augmentation; for validation, only resize and normalize.
train_transform = transforms.Compose([
    transforms.RandomResizedCrop((112, 112), scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),  # converts PIL Image in L mode to tensor shape (1, H, W) with pixel values [0,1]
    transforms.Normalize(mean=[0.5], std=[0.5])
])

val_transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [ ]:
# D) Create Dataset Wrappers

# The HelplessnessVideoDataset loads all frames for a clip,
# converts them to grayscale, applies the provided transform consistently per clip,
# and returns a tensor of shape (T, 1, 112, 112) along with the label.
train_dataset = HelplessnessVideoDataset(train_dir, transform=train_transform)
val_dataset   = HelplessnessVideoDataset(val_dir, transform=val_transform)

# E) DataLoaders
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=0)

# F) Model, Loss, and Optimizer
model = HelplessnessClassifier(num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)


# G) Validation Function
def validate():
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for sequences, labels in val_loader:
            # sequences shape: (B, T, 1, 112, 112)
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = model(sequences)  # Model expects input shape (B, T, 1, 112, 112)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total


# H) Training Loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for sequences, labels in loop:
        # sequences: (B, T, 1, 112, 112)
        sequences, labels = sequences.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(sequences)  # (B, num_classes)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        loop.set_postfix({"loss": running_loss / (loop.n + 1), "acc": correct / total})

    train_acc = (correct / total) * 100
    val_acc = validate() * 100
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1} | Train Loss: {epoch_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")


# I) Save the Model
save_path = os.path.join('.', 'grayscale_cnn_lstm.pth')
torch.save(model.state_dict(), save_path)
print(f"Saved model weights to: {save_path}")

[DEBUG] CNN feature_dim = 25088


Epoch 1/20: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [02:21<00:00,  3.46s/it, loss=1.1, acc=0.404]


Epoch 1 | Train Loss: 1.0980 | Train Acc: 40.37% | Val Acc: 35.90%


Epoch 2/20: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [02:25<00:00,  3.54s/it, loss=1.04, acc=0.453]


Epoch 2 | Train Loss: 1.0418 | Train Acc: 45.34% | Val Acc: 38.46%


...

Epoch 18/20: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [02:38<00:00,  3.87s/it, loss=0.785, acc=0.665]


Epoch 18 | Train Loss: 0.7850 | Train Acc: 66.46% | Val Acc: 46.15%


Epoch 19/20: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [02:46<00:00,  4.07s/it, loss=0.818, acc=0.634]


Epoch 19 | Train Loss: 0.8178 | Train Acc: 63.35% | Val Acc: 41.03%


Epoch 20/20:  83%|██████████████████████████████████████████████████████████████████████████████▊                | 34/41 [02:21<00:28,  4.10s/it, loss=0.749, acc=0.669]